In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization

In [4]:
# 꽃 이미지가 저장된 폴더에서 이미지 파일 가져오기 & 라벨링
# X: 꽃 이미지 파일을 numpy 배열로 변환하여 X 배열에 저장
# Z: 꽃 이미지 마다 라벨(꽃 이름)을 영문자열로 Z 배열에 저장

X=[] #리스트 변수
Z=[]
IMG_SIZE=224
FLOWER_DAISY_DIR='/content/drive/MyDrive/Colab Notebooks/인공지능/12주차/flowers/daisy'
FLOWER_SUNFLOWER_DIR='/content/drive/MyDrive/Colab Notebooks/인공지능/12주차/flowers/sunflower'
FLOWER_TULIP_DIR='/content/drive/MyDrive/Colab Notebooks/인공지능/12주차/flowers/tulip'
FLOWER_DANDI_DIR='/content/drive/MyDrive/Colab Notebooks/인공지능/12주차/flowers/dandelion'
FLOWER_ROSE_DIR='/content/drive/MyDrive/Colab Notebooks/인공지능/12주차/flowers/rose'

In [5]:
from tqdm import tqdm
import os
import cv2

# 폴더에 저장된 이미지 파일들을 Numpy 데이터로 변환
def make_train_data(flower_type, DIR):
    for img in tqdm(os.listdir(DIR)):
      label = flower_type
      path = os.path.join(DIR, img)
      img2 = cv2.imread(path, cv2.IMREAD_COLOR)
      img2 = cv2.resize(img2, (IMG_SIZE, IMG_SIZE))

      X.append(img2)
      Z.append(label)

In [6]:
make_train_data('Daisy',FLOWER_DAISY_DIR)
print(len(X))

make_train_data('Sunflower',FLOWER_SUNFLOWER_DIR)
print(len(X))

make_train_data('Tulip',FLOWER_TULIP_DIR)
print(len(X))

make_train_data('Dandelion',FLOWER_DANDI_DIR)
print(len(X))

make_train_data('Rose',FLOWER_ROSE_DIR)
print(len(X))

100%|██████████| 50/50 [00:10<00:00,  4.91it/s]


50


100%|██████████| 50/50 [00:09<00:00,  5.24it/s]


100


100%|██████████| 50/50 [00:09<00:00,  5.00it/s]


150


100%|██████████| 50/50 [00:09<00:00,  5.18it/s]


200


100%|██████████| 50/50 [00:09<00:00,  5.21it/s]

250


In [7]:
# Z 배열에 저장된 꽃 이름(문자열)을 원핫인코딩 방식으로 변환하여 Y 배열에 저장
# X 배열에 저장된 꽃 이미지는 모든 값을 255로 나누어줌
# 입력 값을 1보다 작은 값으로 만들기 위함

le = LabelEncoder()
tmp = le.fit_transform(Z)
Y = pd.get_dummies(tmp).values

X = np.array(X)
X = X/255.0

In [8]:
# 1. 데이터 준비: 훈련 데이터와 시험 데이터로 분할하기
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((200, 224, 224, 3), (50, 224, 224, 3), (200, 5), (50, 5))

In [ ]:
# 2. 모델 구성하기(AlexNet 구성)
img_shape = (IMG_SIZE, IMG_SIZE, 3)

model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=img_shape, kernel_size=(11, 11),
                 strides=4, padding='same', activation='relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))
# Batch Normalisation before passing it to the next layer
model.add(BatchNormalization())

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(5, 5), strides=1,
                 padding='same', activation='relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3, 3), strides=1,
                 padding='same', activation='relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3, 3), strides=1,
                 padding='same', activation='relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=1,
                 padding='same', activation='relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))
# Batch Normalisation
model.add(BatchNormalization())

# Passing it to a dense layer
model.add(Flatten())
# 1st Dense Layer
model.add(Dense(2048, activation='relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.5))
# Batch Normalisation
model.add(BatchNormalization())

# 2nd Dense Layer
model.add(Dense(1024, activation='relu'))
# Add Dropout
model.add(Dropout(0.5))
# Batch Normalisation
model.add(BatchNormalization())

# Output Layer
model.add(Dense(5, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 56, 56, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 96)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 27, 27, 96)       384       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 256)       614656    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 256)      0         
 2D)                                                             
                                                        

In [ ]:
# 3. 모델 학습과정 설정하기


In [ ]:
# 4. 모델 학습시키기


In [ ]:
# 5. 학습과정 살펴보기
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])

plt.subplot(1, 2, 2)
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
# 6. 학습된 모델로 예측하기


In [ ]:
# 예측결과 올바른 결과 8개(prop_class)와 틀린 결과 8개(mis_class) 선택
i=0
prop_class=[]
mis_class=[]



In [ ]:
# 올바른 예측 결과 보여주기



In [ ]:
# 잘못된 결과 보여주기

